In [1]:
import os
import pandas as pd

data_df = pd.DataFrame(columns=['text','Intent'])
filenames = list()
intervention_subgraphs = list()
for directory, subdirectories, files in os.walk('C:/Users/Tamir/Desktop/HSE_Project/excel_files/'):
    for file in files:
        filenames.append(os.path.join(directory,file))
for filename in filenames:
    if (filename[len(filename) - 22] == '~'):
        continue
    filename = filename.replace(" ", " ")
    post_df = pd.read_excel(filename)
    
        #fix to add columns if they are not in the xlsx file
    if 'attachments' not in post_df.columns:
        post_df['attachments'] = ''
    if 'reply_to_comment' not in post_df.columns:
        post_df['reply_to_comment'] = 0
    if 'reply_to_user' not in post_df.columns:
        post_df['reply_to_user'] = 0
        
    flex_df = post_df.drop(columns=['attachments','date','from_id','id','reply_to_comment','reply_to_user','Content','Intervention'])
    data_df = data_df.append(flex_df,ignore_index = True)

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
def clean_text(text, remove_stopwords=True):
    """
    Removes stop words from the given text
    :param text: string with the text
    :param remove_stopwords: flag to enable removal of stop words
    :return: set of tokens without any stop words
    """
    text = text.lower()
    tokens = tokenizer.tokenize(text)

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words('russian'))
        tokens = [w for w in tokens if w not in stops]

    return tokens

delete_index = list()

for index,row in data_df.iterrows():
    if ( not (pd.isnull(data_df['text'].loc[index]) or pd.isnull(data_df['Intent'].loc[index]))   ):
        
        data_df['text'].loc[index] = clean_text(data_df['text'].loc[index])
    else:
        delete_index.append(index)

data_df = data_df.drop(index = delete_index)
data_df = data_df.reset_index(drop=True)

In [10]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec

documents = [TaggedDocument(data_df['text'].loc[i], data_df['Intent'].loc[i]) for i in range(len(data_df))]
model = Doc2Vec(documents,dm=1, vector_size=5, window=4, min_count=1,epochs=5, workers=4)

In [11]:
model.most_similar('путин')

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('смогут', 0.9990633130073547),
 ('хотя', 0.9984726905822754),
 ('вами', 0.9982699751853943),
 ('х', 0.9980146884918213),
 ('страну', 0.9973703622817993),
 ('войны', 0.9965711236000061),
 ('id370722155', 0.9963447451591492),
 ('школе', 0.9955522418022156),
 ('украину', 0.9950838088989258),
 ('е', 0.9941287040710449)]